In [1]:
import pandas as pd
import numpy as np

Load in `.csv` file that came from `stage-p2x`

In [2]:
df = pd.read_csv('runs_with_eval_loss_and_params.csv')

Load in data from MolNet finetuning

In [3]:
import s3fs
import os

In [4]:
fs = s3fs.S3FileSystem()

In [5]:
import os
import json

In [1]:
model_bucket = ""

In [6]:
cloud_dir = f's3://{model_bucket}/chemberta/mtr_pretraining_77M_20210824/molnet_mtr_77M_ft_20210824/'

In [7]:
def get_dataframes(cloud_dir):
    run_dirs = fs.ls(cloud_dir)
    data_avg = []
    df_all = pd.DataFrame()
    for rd in run_dirs:
        run_name = os.path.basename(os.path.normpath(rd))
        # go one level down to get the molnet task
        molnet_task_data_avg = {}
        molnet_task_data_all = {}
        for molnet_task_dir in fs.ls(rd):
            molnet_task_name = os.path.basename(os.path.normpath(molnet_task_dir))
            results_dir = os.path.join(molnet_task_dir, "results/")
            for subset in ["valid", "test"]:
                with fs.open(os.path.join(results_dir, subset, "metrics.json")) as f:
                    metrics = json.load(f)
                # pick first item to get the keys
                metric_names = list(list(metrics.items())[0][1].keys())
                metric_res = {mn: [] for mn in metric_names}
                for seed, res in metrics.items():
                    for mn, mres in res.items():
                        if mn == "pearsonr":
                            metric_res[mn].append(mres[0])
                        else:
                            metric_res[mn].append(mres)
                molnet_task_data_all.update({f"{molnet_task_name}_{subset}_{mn}": metric_res[mn] for mn in metric_names})
                average_metrics = {f"{molnet_task_name}_{subset}_{mn}_mean": np.mean(metric_res[mn]) for mn in metric_names}
                std_metrics = {f"{molnet_task_name}_{subset}_{mn}_std": np.std(metric_res[mn]) for mn in metric_names}
                molnet_task_data_avg.update({**average_metrics, **std_metrics})
        molnet_task_data_all.update({"run_name": [run_name]*5})
        df_all = df_all.append(pd.DataFrame(molnet_task_data_all))
        data_avg.append({"run_name": run_name, **molnet_task_data_avg})

    df_avg = pd.DataFrame(data_avg)
    return df_all, df_avg

In [8]:
df_all, df_avg = get_dataframes(cloud_dir)

In [9]:
df_all

,bace_classification_valid_roc_auc_score,bace_classification_valid_average_precision_score,bace_classification_test_roc_auc_score,bace_classification_test_average_precision_score,bace_regression_valid_pearsonr,bace_regression_valid_rmse,bace_regression_test_pearsonr,bace_regression_test_rmse,bbbp_valid_roc_auc_score,bbbp_valid_average_precision_score,...,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score,run_name
0,0.553838,0.670666,0.718478,0.775396,0.160819,0.513827,0.746633,1.051594,0.974767,0.973384,...,0.462026,0.777616,0.642299,0.726277,0.632461,0.795084,0.479073,0.756544,0.408131,run_0
1,0.553660,0.670620,0.718478,0.775396,0.157373,0.516597,0.746123,1.079928,0.976611,0.974922,...,0.473172,0.777501,0.638435,0.730301,0.626836,0.810441,0.482046,0.752679,0.398375,run_0
2,0.553838,0.670666,0.718478,0.775396,0.161497,0.515204,0.747100,1.041442,0.973991,0.972021,...,0.461848,0.774194,0.644632,0.729461,0.629643,0.789465,0.465543,0.750966,0.412587,run_0
3,0.553838,0.670666,0.718478,0.775396,0.156009,0.514934,0.748401,1.093741,0.975641,0.974931,...,0.461279,0.771452,0.649541,0.727457,0.630826,0.801261,0.482211,0.750917,0.406049,run_0
4,0.553838,0.670666,0.718478,0.775396,0.157476,0.515940,0.747519,1.069457,0.974767,0.973666,...,0.461746,0.757195,0.663654,0.708210,0.650322,0.785561,0.466375,0.753853,0.417923,run_0
0,0.645522,0.692313,0.812319,0.830578,0.168459,0.524945,0.817728,1.021591,0.973505,0.970600,...,0.426749,0.775774,0.650790,0.731311,0.649791,0.776167,0.495031,0.787465,0.401857,run_1
1,0.643390,0.690766,0.809239,0.828324,0.173298,0.522691,0.816537,0.994048,0.973505,0.970859,...,0.411473,0.760238,0.670921,0.723360,0.651930,0.788650,0.494051,0.791917,0.413548,run_1
2,0.647655,0.694180,0.812138,0.830260,0.170064,0.521427,0.818458,1.009007,0.972535,0.969911,...,0.405471,0.761258,0.667247,0.723565,0.653041,0.767974,0.482813,0.799599,0.415063,run_1
3,0.644279,0.690149,0.813043,0.831212,0.167409,0.523921,0.818498,1.024174,0.972923,0.970117,...,0.408731,0.744768,0.684862,0.714141,0.661010,0.791438,0.482965,0.772836,0.392489,run_1
4,0.642146,0.688018,0.810688,0.828475,0.172563,0.522461,0.817248,0.998296,0.973408,0.970493,...,0.424054,0.772812,0.662174,0.734298,0.653473,0.795942,0.501143,0.784921,0.392068,run_1


In [10]:
df_avg

,run_name,bace_classification_valid_roc_auc_score_mean,bace_classification_valid_average_precision_score_mean,bace_classification_valid_roc_auc_score_std,bace_classification_valid_average_precision_score_std,bace_classification_test_roc_auc_score_mean,bace_classification_test_average_precision_score_mean,bace_classification_test_roc_auc_score_std,bace_classification_test_average_precision_score_std,bace_regression_valid_pearsonr_mean,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_0,0.553802,0.670657,0.000071,0.000018,0.718478,0.775396,0.000000,0.000000e+00,0.158635,...,0.008190,0.008356,0.796362,0.475050,0.008810,0.007511,0.752992,0.408613,0.002091,0.006543
1,run_1,0.644598,0.691085,0.001885,0.002072,0.811486,0.829770,0.001359,1.160975e-03,0.170358,...,0.007048,0.003801,0.784034,0.491201,0.010371,0.007209,0.787348,0.403005,0.008803,0.009880
2,run_38,0.487740,0.551096,0.000000,0.000000,0.643768,0.665333,0.000089,4.081143e-05,0.220960,...,0.010757,0.006533,0.757841,0.446270,0.001440,0.002977,0.819531,0.394257,0.001457,0.001901
3,run_39,0.673454,0.697134,0.001514,0.001792,0.817065,0.840163,0.004224,5.794279e-03,0.169626,...,0.021736,0.016044,0.734626,0.446543,0.009882,0.022625,0.798219,0.404890,0.011476,0.031436
4,run_45,0.585466,0.626927,0.000000,0.000000,0.759058,0.816647,0.000000,1.110223e-16,0.178072,...,0.001592,0.005084,0.776999,0.492551,0.003705,0.002459,0.772504,0.438593,0.003099,0.003468


In [11]:
combined_avg_df = pd.merge(left=df, right=df_avg, on='run_name')
# combined_avg_df['run_name'] = combined_avg_df['run_name'].apply(lambda x: f"mlm_{x}")

combined_all_df = pd.merge(left=df, right=df_all, on='run_name')
# combined_all_df['run_name'] = combined_all_df['run_name'].apply(lambda x: f"mlm_{x}")

In [12]:
combined_avg_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.008190,0.008356,0.796362,0.475050,0.008810,0.007511,0.752992,0.408613,0.002091,0.006543
1,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.001592,0.005084,0.776999,0.492551,0.003705,0.002459,0.772504,0.438593,0.003099,0.003468
2,run_38,0.259809,126,0.109,0.279,456,3,2,0.000021,25,...,0.010757,0.006533,0.757841,0.446270,0.001440,0.002977,0.819531,0.394257,0.001457,0.001901
3,run_1,0.144309,407,0.156,0.295,9044,11,2,0.000005,25,...,0.007048,0.003801,0.784034,0.491201,0.010371,0.007209,0.787348,0.403005,0.008803,0.009880
4,run_39,0.235581,209,0.176,0.128,3968,11,3,0.000002,25,...,0.021736,0.016044,0.734626,0.446543,0.009882,0.022625,0.798219,0.404890,0.011476,0.031436


In [13]:
combined_all_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,delaney_test_pearsonr,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score
0,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.893047,0.462026,0.777616,0.642299,0.726277,0.632461,0.795084,0.479073,0.756544,0.408131
1,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.887858,0.473172,0.777501,0.638435,0.730301,0.626836,0.810441,0.482046,0.752679,0.398375
2,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.892967,0.461848,0.774194,0.644632,0.729461,0.629643,0.789465,0.465543,0.750966,0.412587
3,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.893263,0.461279,0.771452,0.649541,0.727457,0.630826,0.801261,0.482211,0.750917,0.406049
4,run_0,0.070153,264,0.224,0.122,4888,8,3,0.000396,25,...,0.893222,0.461746,0.757195,0.663654,0.708210,0.650322,0.785561,0.466375,0.753853,0.417923
5,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.912385,0.424741,0.824663,0.576621,0.772746,0.594299,0.775995,0.489190,0.770683,0.435602
6,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.911629,0.427028,0.832311,0.570467,0.772040,0.604871,0.770891,0.494648,0.771173,0.442045
7,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.912264,0.423927,0.828891,0.577193,0.769523,0.606575,0.782258,0.493759,0.770488,0.433365
8,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.913492,0.422615,0.827355,0.578347,0.773161,0.602478,0.778612,0.495136,0.778658,0.441514
9,run_45,0.040794,384,0.109,0.144,464,12,3,0.000141,25,...,0.909611,0.431716,0.811002,0.596505,0.769442,0.594937,0.777239,0.490021,0.771515,0.440438


In [14]:
combined_avg_df.to_csv('ft_results_combined.csv', index=False)
combined_all_df.to_csv('ft_results_all_seeds.csv', index=False)